In [1]:
import sys, os
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras

from tensorflow.keras.utils import img_to_array
from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16

In [4]:
train_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/train"
test_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/test"
img_size = 255
num_classes = 3
class_names = ["Crazing", "Inclusion", "Patches"]

In [5]:
augment = keras.Sequential(
    [layers.RandomFlip("horizontal_and_vertical"), 
     layers.RandomRotation(0.2)]
)

2023-11-21 21:41:13.313581: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2023-11-21 21:41:13.313617: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-21 21:41:13.313655: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-21 21:41:13.314338: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-21 21:41:13.315121: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
data_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/"
INPUT_TFRECORD_TRAIN = os.path.join(data_dir, "train_tfrecords")
INPUT_TFRECORD_TEST = os.path.join(data_dir, "test_tfrecords")

In [9]:
class LoadTfrecords:

    def __init__(self, BATCH_SIZE):
        self.BATCH_SIZE = BATCH_SIZE


    def parse_train(self, serialized):
        features = {"image": tf.io.FixedLenFeature([], tf.string), 
                    "label": tf.io.FixedLenFeature([], tf.string)}
        
        parsed = tf.io.parse_single_example(serialized=serialized, 
                                            features=features)
        
        image_raw = parsed["image"]
        label_raw = parsed["label"]

        images = tf.io.decode_raw(image_raw, tf.uint8)
        images = tf.cast(images, tf.float32) / 255
        images = tf.reshape(images, [200, 200, 3])

        labels = tf.io.decode_raw(label_raw, tf.float32)
        labels = tf.reshape(labels, [3])

        return images, labels
    

    def parse_test(self, serialized):
        features = {"image": tf.io.FixedLenFeature([], tf.string), 
                    "label": tf.io.FixedLenFeature([], tf.string)}
        
        parsed = tf.io.parse_single_example(serialized=serialized, 
                                            features=features)
        
        image_raw = parsed["image"]
        label_raw = parsed["label"]

        images = tf.io.decode_raw(image_raw, tf.uint8)
        images = tf.cast(images, tf.float32) / 255
        images = tf.reshape(images, [200, 200, 3])

        labels = tf.io.decode_raw(label_raw, tf.float32)
        labels = tf.reshape(labels, [3])

        return images, labels


In [11]:
parse_data = LoadTfrecords(BATCH_SIZE=32)

trainset = tf.data.TFRecordDataset(INPUT_TFRECORD_TRAIN)
trainset = trainset.map(parse_data.parse_train)
trainset = trainset.shuffle(buffer_size=828)
trainset = trainset.repeat(-1)
trainset = trainset.batch(32)
trainset = trainset.prefetch(buffer_size=4)

testset = tf.data.TFRecordDataset(INPUT_TFRECORD_TEST)
testset = testset.map(parse_data.parse_test)
testset = testset.batch(32)
testset = testset.prefetch(buffer_size=2)

In [2]:
input = Input(shape=(200, 200, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input)

NameError: name 'Input' is not defined

In [1]:
x = vgg16.output
x = Flatten(input_shape=vgg16.output_shape[1:])(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(3, activation='softmax')(x)
model = keras.models.Model(inputs=vgg16.input, outputs=prediction)
model.summary()

NameError: name 'vgg16' is not defined